(sec:exercise-ogura)=
# 演習2 - 百人一首エージェントを作る

今回の演習では、小倉百人一首をプレイヤーAIを作成してみよう。ご存じの通り、小倉百人一首は、100種類の短歌からなるかるたの一種であり、読み札は短歌の上の句 (575の部分)であり、取り札が下の句(77の部分)である。

通常の小倉百人一種では、読み札には上の句と下の句の両方が漢字で書かれており、さらに挿絵がある。一方で取り札は下の句が濁点や半濁点を含まないひらがな46種類によって書かれている。

百人一首では100枚の札のうち50枚だけを並べて、それを2人のプレイヤーによって取り合う。読み札は100枚全てを用いるので、「空札」といって、読まれたけれども50枚の取り札には含まれない札というのも存在する。

細かな百人一首のルールについては、以下の全日本かるた協会のウェブサイトを参考にしてほしい。

**はじめての競技かるた:** <https://www.karuta.or.jp/karuta/first-time/>

## 演習の内容

今回の演習では、取り札が複数含まれた画像と、上の句を表す文字列の先頭数文字が与えられる。例えば、以下のような画像と文字列のペアが与えられる。

**入力画像:**


**入力文字列:**

```
3  
はなのいろは  
あさぼらけ  
ほととぎす
```

この入力に対して、各行のひらがなの列に対して、取り札が一つに決まり、なおかつ、その札が画像に含まれる場合には`1`を、与えられた文字列では札が決まらない場合は`2`を、対応する取り札が存在しない場合には`0`を返すようにする。

例えば、上の入力例では、「はなのいろは...」で始まる句は決まり字が3文字の「はなの」であり、なおかつ下の句である「わかみよにふる なかめせしまに」と書かれた札が画像に映っているので`1`が正解となる。

一方、2つめの「あさぼらけ」は2枚しかない6文字決まり字の札であり、「あさぼらけ ありあけのつきと...」に対応する下の句である「よしののさとに ふれるしらゆき」と、「あさぼらけ うじのかわぎり...」に対応する下の句である「あらはれわたる せせのあしろき」の両方が画像に含まれており、どちらの札を取れば良いかが決まらない。従って正解は`2`となる。

3つ目の「ほとどぎす」は1文字決まりの札であるため、対応する下の句が「たたありあけの つきそのこれる」になることは決定しているが、その下の句の札は画像中に存在しないため、正解は`0`となる。

以上より、上記の入力に対する正しい出力の例は以下のようになる。

**出力:**

```
1
2
0
```

### 問題のレベル

今回の演習では、以下の3つのレベルの入力を用意している。

**レベル1**では、画像に写っている取り札は1枚だけであり、読み札に対応する文字列も1つだけが与えられる。

**レベル2**では、画像に写っている取り札は3枚以上、5枚以下であり、読み札に対応する文字列は1つだけが与えられる。

**レベル3**では、画像に写っている取り札は3枚以上、5枚以下であり、読み札に対応する文字列は3つから5つ与えられる。

これら、各レベルの入力が10枚ずつ与えられ、レベルに対応する点数、すなわちレベル1なら1点、レベル2なら2点、レベル3なら3点が、出力が完璧に正しい場合にのみ与えられる(したがって部分点はない)。

また、全レベル共通で取り札の画像は、任意の方向に回転しており、複数枚の取り札が画像に含まれている場合もその向きは揃っていないものとする。

## 採点方法

今回の演習では、練習用の画像を各レベルにつき5枚ずつ与えるので、それを用いて、演習時間中にプログラムを作成すること。本番の授業の最初には、練習用とは異なる画像を、本番用として各レベルにつき10枚ずつ与えるので、講義時間中に自身のプログラムを修正して、なるべく多くの問題に正解できるようにあらかじめ準備をしておくこと。

## 使用可能なプログラム

今回のプログラムはGoogle Colab上にPyTorchを用いてGPU環境で深層学習を行うことを許可する。ただし、Google Colab上で使うことができないライブラリはPyTorch以外は使わないようにすること。

また、プログラムは必ずしもJupyter Notebook形式で作成する必要はなく、通常のPython形式のファイルで作成しても良い (むしろどちらかというとこちらを推奨)。

最初に断っておくが、今回の課題に関しては、必ずしも深層学習を使う必要はなく、古典的なBoVW+Fisherベクトル+SVMのような手法であっても、十分に良いスコアを取ることができるだろう。

## より高度な問題を解くヒント

小倉百人一首には、上の句にも下の句にも決まり字がある。一例として、

- はなのいろは うつりにけりな いたずらに わかみよにふる なかめせしまに

という句であれば、上の句の決まり字は「はなの」であり、下の句の決まり字は「わかみよ」である。

従って、厳密には、画像に写った読み札に書かれた文字全てを認識する必要はなく、札の右上の方に書かれた数文字だけを認識できれば十分である。もちろん、識別の精度は100%にはならないと思われるので、多めに文字を認識して、間違いがないようにする方が良い。

また、今回の演習では、画像の中に複数の取り札が含まれている可能性がある (その代わり画像中に読み札以外の余計なものをは写っていない)ため、それらを全て画像として取り出す必要がある。しかも札の向きは任意であるため、ひらがなを認識できるような札の向きにどのように揃えるかは一つポイントになるだろう。

今回も、データセットの作成には、[数独の演習](sec:exercise-sudoku)で用いたようなExcelやGoogleスプレッドシートを用いる方法が使える。今回は46文字のひらがなだけが対象であるので、以下の画像のように1行ごとに41文字を並べて、各行を異なるフォントにしてデータセットを作れば良いだろう (やはり、受講者同士で協力してデータセットを作って良い)。

```{image} ./imgs/gen_hira_excel.jpg
:align: center
```

<br/>